In [1]:
from  __future__  import division, print_function, absolute_import

import tensorflow as tf

# Get data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [2]:
tf.test.gpu_device_name()

''

In [3]:
# Some numbers
batch_size = 128
display_step = 10
num_input = 784
num_classes = 10

In [6]:
def conv_layer(inputs, channels_in, channels_out, strides=1):       
        
        # Create variables
        w=tf.Variable(tf.random_normal([3, 3, channels_in, channels_out]))
        b=tf.Variable(tf.random_normal([channels_out]))

        # Define Ops
        x = tf.nn.conv2d(inputs, w, strides=[1, strides, strides, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
        
        # Non-linear activation
        return tf.nn.relu(x)

    
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def fc_layer(x, n_in, n_out, keep_rate=0.8):
    w1=tf.Variable(tf.random_normal([n_in, n_out]))
    b1=tf.Variable(tf.random_normal([n_out]))
    fc1 = tf.add(tf.matmul(x,w1),b1)
    fc1=tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_rate)
    return fc1
    

# Create model
def CNN(x, devices):
    
    keep_rate= 0.8
    with tf.device(devices[0]): # <----------- Put 1 quarter of network on device 0


        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        print("X: ", x.shape)

        # Convolution Layer
        conv1=conv_layer(x, 1, 32, strides=1)
        pool1=maxpool2d(conv1)
        print("1: ", pool1.shape)        

        # Convolution Layer
        conv2=conv_layer(pool1, 32, 64, strides=1)
        pool2=maxpool2d(conv2)
        print("2: ", pool2.shape)
        
        # Fully connected layer
        fc = tf.reshape(pool2, [-1, 7*7*64])
        fc = fc_layer(fc, 7*7*64, 1024, keep_rate=0.8)
        
        for i in range (3):
            # Fully connected layer
            fc = fc_layer(fc, 1024, 1024, keep_rate=0.8)
        
        


    with tf.device(devices[1]):  # <----------- Put 1 quarter of network on device 1

        for i in range (3):
            # Fully connected layer
            fc = fc_layer(fc, 1024, 1024, keep_rate=0.8)
            
    with tf.device(devices[2]): # <----------- Put 1 quarter of network on device 2


            for i in range (3):
                # Fully connected layer
                fc = fc_layer(fc, 1024, 1024, keep_rate=0.8)

    with tf.device(devices[3]):  # <----------- Put 1 quarter of network on device 3

            for i in range (1):
                # Fully connected layer
                fc = fc_layer(fc, 1024, 1024, keep_rate=0.8)
                
            # Output layer
            w2=tf.Variable(tf.random_normal([1024, num_classes]))
            b2=tf.Variable(tf.random_normal([num_classes]))
            out = tf.add(tf.matmul(fc,w2),b2) # finalise
        

    return out

In [7]:
# Define devices that we wish to split our graph over
device0='/job:worker/task:0'
device1='/job:worker/task:1'
device2='/job:worker/task:2'
device3='/job:worker/task:3'

devices=(device0, device1, device2, device3)

tf.reset_default_graph() # Reset graph

# Construct model
with tf.device(devices[0]):
    X = tf.placeholder(tf.float32, [None, num_input]) # Input images feedable
    Y = tf.placeholder(tf.float32, [None, num_classes]) # Ground truth feedable
    
logits = CNN(X, devices) # Unscaled probabilities

with tf.device(devices[3]):
    
    prediction = tf.nn.softmax(logits) # Class-wise probabilities
    
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()

X:  (?, 28, 28, 1)
1:  (?, 14, 14, 32)
2:  (?, 7, 7, 64)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
# List of addreses and ports
IP_ADDRESS1='localhost'
PORT1='2222'
IP_ADDRESS2='localhost'
PORT2='2224'
IP_ADDRESS3='localhost'
PORT3='2226'
IP_ADDRESS4='localhost'
PORT4='2228'

# Define cluster
cluster_spec = tf.train.ClusterSpec({'worker' : [(IP_ADDRESS1 + ":" + PORT1),
                                                 (IP_ADDRESS2 + ":" + PORT2),
                                                 (IP_ADDRESS3 + ":" + PORT3),
                                                 (IP_ADDRESS4 + ":" + PORT4)]})


task_idx=0 # Chief -> task index = 0
server = tf.train.Server(cluster_spec, job_name='worker', task_index=task_idx)

In [9]:
# Check the server definition
server.server_def

cluster {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "localhost:2222"
    }
    tasks {
      key: 1
      value: "localhost:2224"
    }
    tasks {
      key: 2
      value: "localhost:2226"
    }
    tasks {
      key: 3
      value: "localhost:2228"
    }
  }
}
job_name: "worker"
protocol: "grpc"

In [10]:
import psutil
import pandas as pd
import time

In [11]:
print("CPU: ", psutil.cpu_percent())
print("RAM: ", psutil.virtual_memory().percent)

CPU:  3.4
RAM:  61.1


In [12]:
data = {'step': [],
	'loss': [],
    'time': [],
	'acc': [],
    'RAM': [],
	'CPU': []}
df = pd.DataFrame(data)

# Start training
with tf.Session(server.target) as sess:  
    start_time = time.time()

    
    # Run the initializer
    sess.run(init)
    writer = tf.summary.FileWriter("./logs/graph", sess.graph)
    
    for step in range(1000):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
       # print(type(t_s))
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y : batch_y})
            
            new_row = data = {'step': step, 'loss': loss,'time':time.time()-start_time,'acc': acc, 'RAM': psutil.virtual_memory().percent, 'CPU': psutil.cpu_percent()}
            # Append row to the dataframe
            df = df.append(new_row, ignore_index=True)

            print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))
    # Get test set accuracy
    test_acc= sess.run(accuracy, feed_dict={X: mnist.test.images[:256],Y: mnist.test.labels[:256]})
    print("Testing Accuracy: " + str(test_acc))
    print(df)
    

Step 0, Minibatch Loss= 219573434139541504.0000, Training Accuracy= 0.086
Step 1, Minibatch Loss= 170353710222802944.0000, Training Accuracy= 0.102
Step 10, Minibatch Loss= 77989364780498944.0000, Training Accuracy= 0.148
Step 20, Minibatch Loss= 53661025043480576.0000, Training Accuracy= 0.086
Step 30, Minibatch Loss= 37750804026753024.0000, Training Accuracy= 0.062
Step 40, Minibatch Loss= 29853952602275840.0000, Training Accuracy= 0.125
Step 50, Minibatch Loss= 21210554257375232.0000, Training Accuracy= 0.102
Step 60, Minibatch Loss= 20030494812930048.0000, Training Accuracy= 0.125
Step 70, Minibatch Loss= 17590115870179328.0000, Training Accuracy= 0.125
Step 80, Minibatch Loss= 16395538362531840.0000, Training Accuracy= 0.078
Step 90, Minibatch Loss= 13983584587087872.0000, Training Accuracy= 0.117
Step 100, Minibatch Loss= 12047169590657024.0000, Training Accuracy= 0.164
Step 110, Minibatch Loss= 12012823810932736.0000, Training Accuracy= 0.125
Step 120, Minibatch Loss= 1090665176

In [13]:
df.to_csv (r'/home/joking/Studys/6_SOSE2020/ComplexITSystems/Code/exampleServer/data/4_workers_1000_batches.csv', index = False, header=True)

In [ ]:
$tensorboard --logdir=logs